In [1]:
import pandas as pd
import numpy as np
train_data = pd.read_csv("C:/Users/Administrator/Downloads/titanic_kaggle_train.csv")

In [2]:
train_data.isnull().sum()/ train_data.shape[0] *100

PassengerId     0.000000
Survived        0.000000
Pclass          0.000000
Name            0.000000
Sex             0.000000
Age            19.865320
SibSp           0.000000
Parch           0.000000
Ticket          0.000000
Fare            0.000000
Cabin          77.104377
Embarked        0.224467
dtype: float64

In [3]:
name = []
for i in train_data['Name']:
    name.append(i.split(' ')[1])
    
train_data['prefix'] = name

## Imputing Age column

In [4]:
train_data_raw = train_data.drop('Cabin', axis = 1).drop('Name', axis = 1)


a_15 = train_data_raw[(train_data_raw['Age'].isnull() == True) & ((train_data_raw['prefix'] == 'Master.')|(train_data_raw['prefix'] == 'Miss.'))].replace(np.nan, 15)
a_45 = train_data_raw[(train_data_raw['Age'].isnull() == True) & ((train_data_raw['prefix'] != 'Master.')&(train_data_raw['prefix'] != 'Miss.'))].replace(np.nan, 45)

In [5]:
train_data_raw.dropna(axis = 0, inplace=True)

In [6]:
train_data1 = pd.concat([train_data_raw, a_15], axis = 0)

In [7]:
train_data2 = pd.concat([train_data1, a_45], axis = 0)

In [8]:
train_data2.dtypes

PassengerId      int64
Survived         int64
Pclass           int64
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Embarked        object
prefix          object
dtype: object

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier

In [10]:
train_dummy = pd.get_dummies(train_data2.drop('Embarked', axis=1))
train_dummy['Embarked'] = train_data2['Embarked']

In [11]:
train_dummy.columns.tolist()

['PassengerId',
 'Survived',
 'Pclass',
 'Age',
 'SibSp',
 'Parch',
 'Fare',
 'Sex_female',
 'Sex_male',
 'Ticket_110152',
 'Ticket_110413',
 'Ticket_110465',
 'Ticket_110564',
 'Ticket_110813',
 'Ticket_111240',
 'Ticket_111320',
 'Ticket_111361',
 'Ticket_111369',
 'Ticket_111426',
 'Ticket_111427',
 'Ticket_111428',
 'Ticket_112050',
 'Ticket_112052',
 'Ticket_112053',
 'Ticket_112058',
 'Ticket_112059',
 'Ticket_112277',
 'Ticket_112379',
 'Ticket_113028',
 'Ticket_113043',
 'Ticket_113050',
 'Ticket_113051',
 'Ticket_113055',
 'Ticket_113056',
 'Ticket_113059',
 'Ticket_113501',
 'Ticket_113503',
 'Ticket_113505',
 'Ticket_113509',
 'Ticket_113510',
 'Ticket_113514',
 'Ticket_113760',
 'Ticket_113767',
 'Ticket_113773',
 'Ticket_113776',
 'Ticket_113781',
 'Ticket_113783',
 'Ticket_113784',
 'Ticket_113786',
 'Ticket_113787',
 'Ticket_113788',
 'Ticket_113789',
 'Ticket_113792',
 'Ticket_113794',
 'Ticket_113796',
 'Ticket_113798',
 'Ticket_113800',
 'Ticket_113803',
 'Ticket_1138

In [12]:
train, validate = train_test_split(train_dummy, test_size = 0.3)

In [13]:
train_x = train.drop('Embarked', axis=1)
validate_x = validate.drop('Embarked', axis=1)
train_y = train['Embarked']
validate_y = validate['Embarked']

In [14]:
m1 = LogisticRegression()
m2 = DecisionTreeClassifier(max_depth=2)
m3 = RandomForestClassifier(n_estimators=2)
m4 = AdaBoostClassifier()

In [15]:
m1.fit(train_x, train_y)
m2.fit(train_x, train_y)
m3.fit(train_x, train_y)
m4.fit(train_x, train_y)

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=1.0, n_estimators=50, random_state=None)

In [16]:
pred_log = m1.predict(validate_x)
pred_dec = m2.predict(validate_x)
pred_ran = m3.predict(validate_x)
pred_ada = m4.predict(validate_x)

In [17]:
df = pd.DataFrame({'actual':validate_y, 'pred_log':pred_log, 'pred_dec':pred_dec, 'pred_ran':pred_ran, 'pred_ada':pred_ada})

In [18]:
df['stat'] = df['actual'] == df['pred_ada']
df['stat'].value_counts()/df.shape[0]*100

True     70.037453
False    29.962547
Name: stat, dtype: float64

In [19]:
df['stat'] = df['actual'] == df['pred_ran']
df['stat'].value_counts()/df.shape[0]*100

True     67.790262
False    32.209738
Name: stat, dtype: float64

In [20]:
df['stat'] = df['actual'] == df['pred_dec']
df['stat'].value_counts()/df.shape[0]*100

True     73.033708
False    26.966292
Name: stat, dtype: float64

In [21]:
df['stat'] = df['actual'] == df['pred_log']
df['stat'].value_counts()/df.shape[0]*100

True     73.782772
False    26.217228
Name: stat, dtype: float64

In [23]:
test = pd.read_csv("C:/Users/Administrator/Downloads/kaggle_titanic_test.csv")
test_dummy = pd.get_dummies(test)

In [24]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [25]:
name = []
for i in test['Name']:
    name.append(i.split(' ')[1])
    
test['prefix'] = name

In [26]:
test_raw = test.drop('Cabin', axis = 1)


a_15 = test_raw[(test_raw['Age'].isnull() == True) & ((test_raw['prefix'] == 'Master.')|(test_raw['prefix'] == 'Miss.'))].replace(np.nan, 15)
a_45 = test_raw[(test_raw['Age'].isnull() == True) & ((test_raw['prefix'] != 'Master.')&(test_raw['prefix'] != 'Miss.'))].replace(np.nan, 45)

In [27]:
test_raw.dropna(axis = 0, inplace=True)
test_data1 = pd.concat([test_raw, a_15], axis = 0)
test_data2 = pd.concat([test_data1, a_45], axis = 0)

In [28]:
test_dummy = pd.get_dummies(test_data2.drop('Name', axis=1).drop('Ticket', axis=1))

In [45]:
for i in test_dummy.columns.tolist():
    if i not in train_dummy.columns.tolist():
        test_dummy = test_dummy.drop(i, axis=1)

In [46]:
for i in train_dummy.columns.tolist():
    if i not in test_dummy.columns.tolist():
        test_dummy = test_dummy[i] = 0

In [52]:
test_dummy.shape

(417, 721)

In [53]:
train_dummy.shape

(889, 721)

In [51]:
train_x = train.drop('Embarked', axis = 1)
train_y = train['Embarked']
test_x = test_dummy

In [57]:
for i in test_x.columns.tolist():
    if i not in train_x.columns.tolist():
        test_x = test_x.drop(i, axis=1)

In [58]:
for i in train_x.columns.tolist():
    if i not in test_x.columns.tolist():
        test_x = test_x[i] = 0

In [68]:
model = LogisticRegression()
model.fit(train_x, train_y)
prediction = model.predict(test_x)